In [1]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
FICHERO_DATA = 'data/data_reviews.csv'

In [4]:
df = pd.read_csv(FICHERO_DATA)

In [17]:
df.head()

,reseñas,IA,longitud,longitud_promedio,palabras_unicas,signos_de_puntuación,frecuencia_pronombres,variedad_lexica,entropia_lexica,expresiones,tfidf,palabras_complejas,tecnicismos,polaridad,subjetividad,coherencia
0,Necesitaba un nuevo movil y me decidí por este...,0,297,4.066667,0.733333,6,0,0.733333,5.267482,0,5.640761e-02,3,0,0.0,0.0,0.920000
1,Está en perfectas condiciones la batería al 10...,0,99,5.375000,1.000000,2,0,1.000000,4.000000,0,5.551115e-17,2,0,0.0,0.0,1.000000
2,Me llegó ayer el iPhone en perfectas condicion...,0,436,4.402439,0.695122,7,0,0.707317,5.710972,0,5.749596e-02,5,0,0.0,0.0,0.857143
3,Compré un iPhone 12 a este vendedor y hace un...,0,657,4.124031,0.581395,10,1,0.604651,5.880116,0,7.866846e-02,5,0,0.0,0.0,0.823529
4,Soy usuario de IPhone desde el principio. Teng...,0,596,4.109244,0.638655,11,0,0.655462,5.953860,0,7.357145e-02,4,0,0.0,0.0,0.953488


In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

# Preprocesar el texto
df['reseñas'] = df['reseñas'].str.lower()  # Convertir a minúsculas
# Puedes añadir más pasos de preprocesamiento aquí según sea necesario

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df['reseñas'], df['IA'], test_size=0.2, random_state=42)



In [22]:
# Tokenización con BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenización del conjunto de entrenamiento y prueba
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, max_length=128)


c:\Users\pabma\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [23]:
import tensorflow as tf

# Convertir etiquetas a tensores
train_labels = tf.convert_to_tensor(y_train.tolist())
test_labels = tf.convert_to_tensor(y_test.tolist())

# Crear datasets de TensorFlow
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).batch(16)
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
)).batch(16)


In [24]:
from transformers import TFBertForSequenceClassification

# Definir el modelo
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Compilar el modelo
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])

# Entrenar el modelo
model.fit(train_dataset, epochs=3, batch_size=16, validation_data=test_dataset)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: Could not interpret optimizer identifier: <keras.src.optimizers.adam.Adam object at 0x00000159C1B7D280>

In [25]:
loss, accuracy = model.evaluate(test_dataset)
print(f'Accuracy: {accuracy}')


RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.